# GPT-3.5 Classifier 

## Arousal (with examples)

In [19]:
import pandas as pd

In [30]:
# set directory to get .env file. You will need to change this to wherever your API key (.env file) is, as well as where you want to save your results
%cd "/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2" 

/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2


In [31]:
df=pd.read_csv('/Users/stevenmesquiti/Desktop/Ardens-Project/mindcore-research/GPT-stuff/master_text.csv') # load in the data using pandas and take a peek at the first 10 rows
df.head(10) #grab the text column 

,Unnamed: 0,SID,value,arousal,tone
0,2609,s348,It's always goof to have moral support to trea...,NaN,NaN
1,2369,s623,we should have responsible for keep our wilds ...,NaN,NaN
2,1177,s622,A proposal to reduce the 28 country bloc's net...,NaN,NaN
3,1098,s496,"to prevent the plants , its god's gift",NaN,NaN
4,1252,s083,"As the climate changes, more heritage sites ar...",NaN,NaN
5,3218,s062,With climate change currently happening and ou...,NaN,NaN
6,634,s371,"My sexual function may be getting worse, but n...",NaN,NaN
7,2097,s153,Bananas may be healthier than the sports drink...,NaN,NaN
8,1152,s581,These vegan foods are good for health,NaN,NaN
9,1327,s195,Climate change is a scam and so are the people...,NaN,NaN


In [32]:
# Read in your examples as `examples` and take a peek at the dataframe
examples=pd.read_csv('/Users/stevenmesquiti/Desktop/Ardens-Project/mindcore-research/GPT-stuff/arousal-rating-ex.csv', index_col=0)
examples.head(10)

,text,rating
text number,,
1,Information about how to switch to a renewable...,1
2,Seems to be a rundown of an interview with an ...,1
3,Court blocks Exxon's attempt to stop the inves...,2
4,The article is about weight loss based on diet...,2
5,Because I am a woman and sharing this article ...,3
6,"It matters to people I know, because they care...",4
7,I am very interested with this article so i fe...,4
8,"This is primarily an attack on my home state, ...",5
9,We worry a lot about the future when it comes ...,5


In [33]:
inputs = df.value.values #save the text as an object called inputs. we'll use this to run our observations through the pipeline

In [34]:
# View the first 5 text values to check everything out 
print(inputs[:5])

["It's always goof to have moral support to treat eating disorders"
 'we should have responsible for keep our wilds from fire free.'
 "A proposal to reduce the 28 country bloc's net carbon emission failed"
 "to prevent the plants , its god's gift"
 'As the climate changes, more heritage sites are being affected.']


In [35]:
examples.text.iloc[2]

"Court blocks Exxon's attempt to stop the investigation into the company's role in climate change."

In [36]:
#load other dependencies 
import argparse 
import json
import os
import numpy as np
import openai
import pandas as pd
import random
import time
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import time

In [37]:
title='BBPRIME'
subtitle='Arousal'
stim_set=title+'-'+subtitle
seed=1 #set seed to make sure we get reproducable results 
temperature=0.0 #want a low baking temp to have little variability or creativity, and so we can get the same results every. single. time. range is 0-1
engine='gpt-4' #change this to use different models available to you via OpenAI
n_context=1
cache = True   
resume=False
# MIDI='freq' #or 'name', 'number', 'freq'
audience='People'
item='Text'

load_dotenv()

apiKey = os.environ.get('steven_key') #for stevens key. update to you whatever you're calling your api_key. it should look like this in your file

# apiKey = sk-ZJ...................4T

openai.api_key = apiKey #installize the key so we can access

if cache:
    random.seed(seed)

In [38]:
def generate_prompt(current):#(example_idxs, current_pair):
    prompt = f"""Your task is to classify a piece of as having either high or low arousal. Arousal is the extent to which a stimulus is calming or exciting. Examples of emotions with high
arousal are anger, joy, fear, surprise, or anxiety. Examples of emotions with low arousal are sadness, relaxation, disgust, content, or peace.

You will be provided with 2 examples of high arousal text and 2 examples of low arousal. Answer with only a whole number betwen 1 and 5, with 1 being
 low arousal and 5 being high arousal.


Text: {examples.text.iloc[0]}  
Rating: {examples.rating.iloc[0]}
    
Text: {examples.text.iloc[2]}
Rating: {examples.rating.iloc[2]}

Text: {examples.text.iloc[5]}
Rating: {examples.rating.iloc[5]}

Text: {examples.text.iloc[7]}
Rating: {examples.rating.iloc[7]}
    
Text: {inputs[current]}
Rating:
"""
    if n_context==0:
        prompt = f"""What is the arousal of following piece of text? Answer only with a number between 1 and 5: 1 if low arousal, and 5 if high arousal Here is the text:\n{inputs[current]}"""
    
    return prompt

cache_folder = f'cache/{stim_set}'
os.makedirs(cache_folder, exist_ok=True)
def create_cache_filename():
    filename = f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}'
    # if args.shuffle_context_each_draw:
    #     filename += '-shuffle'
    return os.path.join(cache_folder, filename + '.json')

if not resume:
    visited = []
    predicted_ratings = np.zeros((len(inputs)))
    request_count = 0

cache = {}

if cache and os.path.exists(create_cache_filename()):
    cache = json.load(open(create_cache_filename()))
cached_keys = list(cache.keys())

for idx1, bname1 in enumerate(tqdm(inputs)):
    current = idx1
    if current in visited and predicted_ratings[current]!=0:
        print(f'Already visited {current}')
        continue

    visited.append(current)

    key = f'{current}'
    if key in cached_keys:
        choices = cache[key]['choices']
        print('Using cached choices for key', key)
    else:
        prompt = generate_prompt(current)#(example_idxs, current_pair)
        response=False
        i=0
        while not response:
            i+=1
            try:
                response = openai.ChatCompletion.create(
                    model=engine,
                    messages=[{'role':'user', 'content':prompt}],
                    temperature=temperature,
                    timeout=10
                )
            except Exception as e:
                print(f'Attempt {i} failed')
                time.sleep(5)
        choices = [dict(choice.items()) for choice in response.choices]

        cache[key] = {
            'prompt': prompt,
            'choices': choices,
            'created': response.created
        }

        request_count += 1
        if cache and request_count % 5 == 0:
            json.dump(cache, open(create_cache_filename(), 'w'))

    try:
        answer = choices[0]['message']['content'].replace('\n', '').strip()
        predicted_ratings[idx1] = int(answer)
    except:
        print('Error', cache[key])

os.makedirs(f'predictions/{stim_set}', exist_ok=True)
np.save(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', predicted_ratings)


  0%|          | 0/400 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [35]:
df=pd.read_csv('/Users/stevenmesquiti/Desktop/Ardens-Project/mindcore-research/GPT-stuff/master_text.csv') # load in the data using pandas and take a peek at the first 10 rows
predicted_ratings=np.load(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', allow_pickle=True)
df.head()

,Unnamed: 0,SID,value,arousal,tone
0,2609,s348,It's always goof to have moral support to trea...,NaN,NaN
1,2369,s623,we should have responsible for keep our wilds ...,NaN,NaN
2,1177,s622,A proposal to reduce the 28 country bloc's net...,NaN,NaN
3,1098,s496,"to prevent the plants , its god's gift",NaN,NaN
4,1252,s083,"As the climate changes, more heritage sites ar...",NaN,NaN


In [36]:
predicted_ratings # just to check it ran smoothly
df.columns

Index(['Unnamed: 0', 'SID', 'value', 'arousal', 'tone'], dtype='object')

In [38]:
arousal_df_gpt35_250=pd.DataFrame(predicted_ratings, columns=['gpt_arousal_ratings'])
arousal_df_gpt35_250['value']=df.value.values #update to whatever the txt column is called
arousal_df_gpt35_250['SID']=df.SID.values #update to SID column 

In [39]:
#save the work
output_directory = '/Users/stevenmesquiti/Desktop/Ardens-Project/mindcore-research/GPT-stuff/' #update to where you want to save the work
os.makedirs(output_directory, exist_ok=True)

arousal_df_gpt35_250.to_csv(os.path.join(output_directory, f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}.csv'), index=False) #saving the pandas dataframe to a csv

## Emotional Tone (with examples)

In [12]:
import pandas as pd
    # set directory to get .env file

In [13]:
# set directory to get .env file. You will need to change this to wherever your API key (.env file) is, as well as where you want to save your results
%cd "/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2" 

/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2


In [14]:
df=pd.read_csv('/Users/stevenmesquiti/Desktop/Ardens-Project/mindcore-research/GPT-stuff/master_text.csv') # load in the data using pandas and take a peek at the first 10 rows
df.head(10) #grab the text column 

,Unnamed: 0,SID,value,arousal,tone
0,2609,s348,It's always goof to have moral support to trea...,NaN,NaN
1,2369,s623,we should have responsible for keep our wilds ...,NaN,NaN
2,1177,s622,A proposal to reduce the 28 country bloc's net...,NaN,NaN
3,1098,s496,"to prevent the plants , its god's gift",NaN,NaN
4,1252,s083,"As the climate changes, more heritage sites ar...",NaN,NaN
5,3218,s062,With climate change currently happening and ou...,NaN,NaN
6,634,s371,"My sexual function may be getting worse, but n...",NaN,NaN
7,2097,s153,Bananas may be healthier than the sports drink...,NaN,NaN
8,1152,s581,These vegan foods are good for health,NaN,NaN
9,1327,s195,Climate change is a scam and so are the people...,NaN,NaN


In [39]:
# Read in your examples as `examples` and take a peek at the dataframe
examples=pd.read_csv('/Users/stevenmesquiti/Desktop/Ardens-Project/mindcore-research/GPT-stuff/tone-rating-ex.csv', index_col=0)
examples.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/stevenmesquiti/Desktop/Ardens-Project/mindcore-research/GPT-stuff/tone-rating-ex.csv'

In [47]:
inputs = df.value.values #save the text
print(inputs[:3])

["It's always goof to have moral support to treat eating disorders"
 'we should have responsible for keep our wilds from fire free.'
 "A proposal to reduce the 28 country bloc's net carbon emission failed"]


In [16]:
#load other dependencies 
import argparse
import json
import os
import numpy as np
import openai
import pandas as pd
import random
import time
from tqdm.notebook import tqdm
from dotenv import load_dotenv

In [17]:
title='BBPRIME'
subtitle='Arousal'
stim_set=title+'-'+subtitle
seed=1 #set seed to make sure we get reproducable results 
temperature=0.0 #want a low baking temp to have little variability or creativity, and so we can get the same results every. single. time. range is 0-1
engine='gpt-4' #change this to use different models available to you via OpenAI
n_context=1
cache = True   
resume=False
# MIDI='freq' #or 'name', 'number', 'freq'
audience='People'
item='Text'

load_dotenv()

apiKey = os.environ.get('steven_key') #for stevens key. update to you whatever you're calling your api_key. it should look like this in your file

# apiKey = sk-ZJ...................4T

openai.api_key = apiKey #installize the key so we can access

if cache:
    random.seed(seed)

In [18]:
def generate_prompt(current):#(example_idxs, current_pair):
    prompt = f"""Your task is to classify the emptional tone of a piece of text positive or negative. Emotional tone is the degree of positivity or negativity of a text. Positive tone has a more upbeat
style and can include words like “good, well, new, love” but is not limited to these words. Negative tone reveals anxiety, sadness, and hostility and can include words like “bad, wrong, hate, too much” but is not limited to these words.
You will be provided with 2 examples of text with a positive tone and 2 examples of text with a negative tone. Answer with only a number, with 0 being negative tone and 1 being positive tone


Text: {inputs[current]}
Rating:
"""
    if n_context==0:
        prompt = f"""Does this text have a positive or negative tone? Answer with only a number, with 0 being negative tone and 1 being positive tone. Here is the text:\n{inputs[current]}"""
    
    return prompt

cache_folder = f'cache/{stim_set}'
os.makedirs(cache_folder, exist_ok=True)

def create_cache_filename():
    filename = f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}'
    # if args.shuffle_context_each_draw:
    #     filename += '-shuffle'
    return os.path.join(cache_folder, filename + '.json')

if not resume:
    visited = []
    predicted_ratings = np.zeros((len(inputs)))
    request_count = 0

cache = {}

if cache and os.path.exists(create_cache_filename()):
    cache = json.load(open(create_cache_filename()))
cached_keys = list(cache.keys())

for idx1, bname1 in enumerate(tqdm(inputs)):
    current = idx1
    if current in visited and predicted_ratings[current] != 0:
        print(f'Already visited {current}')
        continue

    visited.append(current)

    key = f'{current}'
    if key in cached_keys:
        choices = cache[key]['choices']
        print('Using cached choices for key', key)
    else:
        prompt = generate_prompt(current)
        response = False
        i = 0
        while not response:
            i += 1
            try:
                response = openai.ChatCompletion.create(
                    model=engine,
                    messages=[{'role': 'user', 'content': prompt}],
                    temperature=temperature,
                    timeout=10
                )
            except Exception as e:
                print(f'Attempt {i} failed\n{e}\n')
                
                time.sleep(7) #7 second delay 
        choices = [dict(choice.items()) for choice in response.choices]

        cache[key] = {
            'prompt': prompt,
            'choices': choices,
            'created': response.created
        }

        request_count += 1
        if cache and request_count % 5 == 0:
            json.dump(cache, open(create_cache_filename(), 'w'))

    try:
        answer = choices[0]['message']['content'].replace('\n', '').strip()
        predicted_ratings[idx1] = int(answer)
    except Exception as e:
        print(f'Error: {e}\n\n{cache[key]}')

os.makedirs(f'predictions/{stim_set}', exist_ok=True)
np.save(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', predicted_ratings)


  0%|          | 0/400 [00:00<?, ?it/s]

Attempt 1 failed
The model `gpt-4` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.

Attempt 2 failed
The model `gpt-4` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.



KeyboardInterrupt: 

In [53]:
df=pd.read_csv("/Users/stevenmesquiti/Desktop/Ardens-Project/mindcore-research/GPT-stuff/master_text.csv") #read in the OG dataframe again
predicted_ratings=np.load(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', allow_pickle=True)
df.head()

,Unnamed: 0,SID,value,arousal,tone
0,2609,s348,It's always goof to have moral support to trea...,NaN,NaN
1,2369,s623,we should have responsible for keep our wilds ...,NaN,NaN
2,1177,s622,A proposal to reduce the 28 country bloc's net...,NaN,NaN
3,1098,s496,"to prevent the plants , its god's gift",NaN,NaN
4,1252,s083,"As the climate changes, more heritage sites ar...",NaN,NaN


In [54]:
predicted_ratings # just to check it ran smoothly
df.columns

Index(['Unnamed: 0', 'SID', 'value', 'arousal', 'tone'], dtype='object')

In [55]:
tone_df_gpt35=pd.DataFrame(predicted_ratings, columns=['tone_ratings'])
tone_df_gpt35['value']=df.value.values #update with text column 
tone_df_gpt35['SID']=df.SID.values #update with indexing variable 

In [ ]:
#save the work
output_directory = '/Users/stevenmesquiti/Desktop/Ardens-Project/mindcore-research/GPT-stuff/tone-rating-ex.csv' #update to your own directory 
os.makedirs(output_directory, exist_ok=True)
tone_df_gpt35.to_csv(os.path.join(output_directory, f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}.csv'), index=False)